# Intro
An over the weekend work with the old dataset, the work is to be continued and I will update in due time. I would love some feedback on the analysis and the code, some parts of the code are pretty 'hacky' and I have to rewrite them better.

If you find the following analysis helpful, some upvotes will be appreciated.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import re
from nltk.stem import PorterStemmer
from nltk import word_tokenize
from nltk.corpus import stopwords 
from nltk.collocations import *
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
%matplotlib inline

In [ ]:
wine = pd.read_csv('../input/winemag-data_first150k.csv', encoding='utf-8')
wine.drop(['Unnamed: 0', 'region_2'], axis = 1, inplace = True)

In [ ]:
print('Shape: ', wine.shape)
print('=========================================================================')
print(wine.head())

# Initial checks

## Checking for missing values
There appears to be missing values in the data. Let's see which columns have missing values and how many there are

In [ ]:
data = []
for col in wine.columns:
    data.append([col, wine[col].isnull().sum(), '{:.2f}%'.format(float(wine[col].isnull().sum())/len(wine[col])*100) ])
df = pd.DataFrame(data, columns=['columns', 'missing value count', 'missing value percentage'])
df

## Checking for duplicate rows

In [ ]:
duplicated_records = wine[wine.duplicated(['country', 'description', 'designation', 'points', 'price', 'province', 'region_1', 'variety', 'winery'])]
duplicated_records.shape

# Preprocessing
## Drop duplicates and remove missing values

In [ ]:
# drop all duplicates and check how many left
new_wine = wine.drop_duplicates()
new_wine.shape

In [ ]:
# check again for missing values
data = []
for col in new_wine.columns:
    data.append([col, new_wine[col].isnull().sum(), '{:.2f}%'.format((new_wine[col].isnull().sum())/len(new_wine[col])*100) ])
df = pd.DataFrame(data, columns=['columns', 'missing value count', 'missing value percentage'])
df

In [ ]:
# I will remove only the three rows in the dataset that have missing values in the country and province column
new_wine = new_wine.dropna(subset=['country', 'province'])
new_wine.shape

# Let the exploration begin
## What is the distribution of the wine points?

In [ ]:
fig, ax = plt.subplots(figsize = (10, 7))
ax = sns.distplot(new_wine['points'], bins = 20, kde=False)

**A few things to note:**
    - The majority of the wines are rated between 86 and 90 points. 
    - The mean value of the points is about 88
    - There are 11 wines that have been rated 100 points and 28 wines that have been rated 99

### Just out of curiosity, let's see where do the wines that are rated 100 points come from

In [ ]:
wines100 = new_wine[new_wine['points'] == 100]
wines100['country'].value_counts()

### And let's see the average price

In [ ]:
wines100.groupby('country')['price'].mean()

### Conclusion:
    - US seems to have more value for money wines with highly rated wines but not so high prices
    - France has few highly rated wines but with high prices
    - Italy and Australia are somewhere in the middle

# Which countries and which province produce expensive wine
Here I am not looking at which country and province produces the most expensive wine, but instead I am looking for the country and province that produces more pricey wines comparing to other.

I will find the median price of wine per country because I don't want my distribution to be affected by the ridiculously expensive wines.

In [ ]:
price_by_country_and_province = new_wine.groupby(['country', 'province'])['price'].median()
df_price = pd.DataFrame(price_by_country_and_province)
df_price.reset_index(inplace = True)
df_price.head()

In [ ]:
mean_price_in_country = df_price.groupby(['country'])['price'].mean()
df_mean_price = pd.DataFrame(mean_price_in_country)
df_mean_price.reset_index(inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 8))
ax = sns.barplot(x = 'country', y = 'price', data = df_mean_price)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_title('Distribution of countries whose provinces produce expensive wine');

In [ ]:
max_price_in_country = df_price.groupby(['country'])['price'].max()
df_max_price = pd.DataFrame(mean_price_in_country)
df_max_price.reset_index(inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 8))
ax = sns.barplot(x = 'country', y = 'price', data = df_max_price)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
ax.set_title('Distribution of countries with expensive provinces');

### Conclusion:
    - From the first chart, I was looking for the country which, generally, has expensive wines. It seems that 
    US-France ranks the highest, followed by England and then Luxembourg.
    - On the second chart, I was looking for which country has a province that produces high priced wines. As a result, these countries are: Chile, Portugal, Germany (didn't expect that) and US.

# Which province produces high quality wine
This is the same as above here I am looking at the provinces in respect to the points

In [ ]:
points_by_country_and_province = new_wine.groupby(['country', 'province'])['points'].mean()
split_points_test = pd.DataFrame(points_by_country_and_province)
split_points_test.reset_index(inplace = True)
split_points_test.head()

In [ ]:
t = split_points_test.groupby(['country'])['points'].max()
x = pd.DataFrame(t)
x.reset_index(inplace = True)

In [ ]:
fig, ax = plt.subplots(figsize = (15, 8))
ax = sns.barplot(x = 'country', y = 'points', data = x)
ax.set_xticklabels(ax.get_xticklabels(), rotation=90);

## Conclusion:
    -Although a bit hard to read, it seems that New Zealand, England, Chile and Portugal receive high points for their wines.

# Analysis of the description of the wine
I start with a simple exploratory analysis of popular words and phrases just to get a feeling of the data. Then I will try to identify which words are important characteristics of a wine.

In [ ]:
def normalize(text):
    letters = re.sub("[^a-zA-Z]", " ", text)
    words = letters.lower()
    return words

In [ ]:
processed_descriptions = []
for description in new_wine['description']:
    norm = normalize(description)
    processed_descriptions.append(norm)

In [ ]:
tfidf_vectorizer = TfidfVectorizer(stop_words='english', ngram_range=(1,2), min_df=3)
tfidf_matrix = tfidf_vectorizer.fit_transform(processed_descriptions)

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()
doc = 10
feature_index = tfidf_matrix[:,:].nonzero()[1]
tfidf_scores = zip(feature_index, sorted([tfidf_matrix[doc, x] for x in feature_index]))

In [ ]:
# function were taken from here: https://buhrmann.github.io/tfidf-analysis.html
def top_tfidf_feats(row, features, top_n=10):
    ''' Get top n tfidf values in row and return them with their corresponding feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return top_feats

def top_feats_in_doc(Xtr, features, row_id, top_n=10):
    ''' Top tfidf features in specific document (matrix row) '''
    row = np.squeeze(Xtr[row_id].toarray())
    return top_tfidf_feats(row, features, top_n)

In [ ]:
# limit dataset to the more popular wine varieties
df_variety = new_wine.groupby('variety').filter(lambda x: len(x) > 5000)

In [ ]:
word_scores = []
for i, j in enumerate(df_variety['variety']):
    for item in top_feats_in_doc(tfidf_matrix, feature_names, i):
        word_scores.append(item + (j,))

In [ ]:
df = pd.DataFrame(word_scores)
df.columns = ['feature', 'tfidf', 'variety']
df = df.sort_values('tfidf', ascending=False).groupby('variety').head(12)

In [ ]:
sns.factorplot(x = 'tfidf', y = 'feature', col='variety', col_wrap=2, data = df, kind='bar', size=10, aspect=.5)

In [ ]:
df_country = new_wine.groupby('country').filter(lambda x: len(x) > 3000)

In [ ]:
word_scores_by_country = []
for i, j in enumerate(df_country['country']):
    for item in top_feats_in_doc(tfidf_matrix, feature_names, i):
        word_scores_by_country.append(item + (j,))

In [ ]:
df = pd.DataFrame(word_scores_by_country)
df.columns = ['feature', 'tfidf', 'country']
df = df.sort_values('tfidf', ascending=False).groupby('country').head(12)

In [ ]:
sns.factorplot(x = 'tfidf', y = 'feature', col='country', col_wrap=2, data = df, kind='bar', size=10, aspect=.5)